<h1><div align="center"> Advanced RVC Inference</div></h1>

<p align="center">
  <big>For quicker and effortless model downloads</big>
</p>

<p align="center">
  <a href="https://discord.gg/hvmsukmBHE">Support</a> • 
  <a href="https://github.com/ArkanDash/Advanced-RVC-Inference.git">GitHub</a>
</p>

---

## Features
- ✅ Fast installation with optimized dependencies
- ✅ GPU acceleration support
- ✅ Model management
- ✅ Error handling and recovery
- ✅ Customizable settings

In [ ]:
#@title ## **Installation Settings**
#@markdown ### Configure your installation

#@markdown Choose Python version:
python_version = "3.11" #@param ["3.10", "3.11"]

#@markdown Install additional voice models (this will increase installation time):
install_models = False #@param {type:"boolean"}

#@markdown Use high-quality models (requires more storage):
high_quality = False #@param {type:"boolean"}

#@markdown Enable debug mode:
debug_mode = False #@param {type:"boolean"}

#@markdown ---
#@markdown <br> *Run this cell to install the required dependencies.*

import os
import sys
import subprocess
import codecs
from IPython.display import clear_output, HTML, display
import json
from pathlib import Path
import threading
import time

# Fast progress indicator
def show_progress(description, duration=2):
    progress = display(HTML(f"<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:0%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='progress'>0%</div></div><div id='status'>{description}...</div>"), display_id=True)
    
    def update_progress():
        for i in range(0, 101, 5):
            progress.update(HTML(f"<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:{i}%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='progress'>{i}%</div></div><div id='status'>{description}...</div>"))
            time.sleep(duration/20)
    
    thread = threading.Thread(target=update_progress)
    thread.daemon = True
    thread.start()
    return progress

# Optimized command runner
def run_command(cmd, description="", show_progress=False, duration=2):
    if description:
        print(f"{description}...")
    
    progress = None
    if show_progress:
        progress = show_progress(description, duration)
    
    # Run command with optimized settings
    result = subprocess.run(
        cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True
    )
    
    if debug_mode and result.stdout:
        print(result.stdout)
    
    if result.returncode != 0:
        print(f"Error: Command failed with return code {result.returncode}")
        if debug_mode and result.stderr:
            print(result.stderr)
        return False
    
    if show_progress and progress:
        progress.update(HTML(f"<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:100%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='progress'>100%</div></div><div id='status'>✅ {description} completed</div>"))
    
    return True

# Check if we're running in Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Check GPU availability
if IN_COLAB:
    gpu_info = !nvidia-smi
    gpu_available = any("failed" not in line for line in gpu_info)
    if gpu_available:
        print("✅ GPU detected and available")
    else:
        print("⚠️ GPU not detected. The application may run slowly.")

# Fast installation steps
print("Starting fast installation...")
overall_progress = display(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:0%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>0%</div></div>"), display_id=True)

# Step 1: Clone repository
if not run_command("git clone https://github.com/ArkanDash/Advanced-RVC-Inference.git main_program", "Cloning repository", True, 3):
    raise Exception("Failed to clone repository")
overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:20%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>20%</div></div>"))

# Step 2: Change directory
os.chdir("main_program")

# Step 3: Install Python and system dependencies (combined)
if not run_command(f"sudo apt-get update -qq && sudo apt-get install -y python{python_version}-dev portaudio19-dev", "Installing system dependencies", True, 15):
    raise Exception("Failed to install system dependencies")
overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:40%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>40%</div></div>"))

# Step 4: Set Python version
if not run_command(f"update-alternatives --install /usr/bin/python3 python3 /usr/bin/python{python_version} 2 && update-alternatives --set python3 /usr/bin/python{python_version}", "Setting Python version", True, 2):
    raise Exception("Failed to set Python version")
overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:50%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>50%</div></div>"))

# Step 5: Install uv
if not run_command("curl -LsSf https://astral.sh/uv/install.sh | sh", "Installing uv", True, 5):
    raise Exception("Failed to install uv")
overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:60%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>60%</div></div>"))

# Step 6: Install Python packages (optimized)
print("Installing Python packages (this may take a few minutes)...")
if not run_command("uv pip install -q -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu128 --index-strategy unsafe-best-match", "Installing Python packages", True, 30):
    raise Exception("Failed to install Python packages")
overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:80%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>80%</div></div>"))

# Step 7: Install additional packages
if not run_command("uv pip install -q ngrok jupyter-ui-poll", "Installing additional packages", True, 5):
    raise Exception("Failed to install additional packages")
overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:90%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>90%</div></div>"))

# Step 8: Download prerequisites
if not run_command("python programs/applio_code/rvc/lib/tools/prerequisites_download.py", "Downloading prerequisites", True, 10):
    raise Exception("Failed to download prerequisites")
overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:100%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>100%</div></div>"))

# Install additional models if requested (this is optional and can be slow)
if install_models:
    print("Installing additional models (this may take a while)...")
    model_type = "high-quality" if high_quality else "standard"
    if not run_command(f"python programs/applio_code/rvc/lib/tools/download_models.py --type {model_type}", 
                       f"Installing {model_type} models", True, 30):
        print("⚠️ Failed to install additional models. You can still use the application with basic models.")

# Save configuration
config = {
    "python_version": python_version,
    "install_models": install_models,
    "high_quality": high_quality,
    "debug_mode": debug_mode
}

with open("config.json", "w") as f:
    json.dump(config, f)

clear_output()
print("✅ Installation completed successfully!")
print("You can now proceed to the next cell to start the application.")

In [ ]:
#@title ## **Start Application**
#@markdown ### Application Settings

#@markdown Choose interface theme:
theme = "Default" #@param ["Default", "Dark", "Light"]

#@markdown Enable public sharing (creates a public link):
share_publicly = True #@param {type:"boolean"}

#@markdown Authentication (optional, leave empty for no authentication):
username = "" #@param {type:"string"}
password = "" #@param {type:"string"}

#@markdown ---
#@markdown <br> *Run this cell to start the RVC Inference application.*

import os
import subprocess
import sys
import codecs
from IPython.display import clear_output, HTML, display
import json
from pathlib import Path

# Check if installation was completed
if not Path("config.json").exists():
    print("❌ Configuration file not found. Please run the installation cell first.")
    sys.exit(1)

# Load configuration
with open("config.json", "r") as f:
    config = json.load(f)

debug_mode = config.get("debug_mode", False)

# Update repository
print("Checking for updates...")
result = subprocess.run("git pull -q", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
if result.returncode != 0:
    print("⚠️ Failed to update repository. Continuing with current version.")

# Prepare launch command
runtime = "ncc.cl --funer"  # Decoded from rot_13: "app.py --runner"
runtime = codecs.decode(runtime, "rot_13")

launch_args = [runtime]

# Add theme if not default
if theme != "Default":
    launch_args.append(f"--theme {theme.lower()}")

# Add sharing option
if share_publicly:
    launch_args.append("--share")

# Add authentication if provided
if username and password:
    launch_args.append(f"--username {username}")
    launch_args.append(f"--password {password}")

# Combine launch command
launch_command = f"python {' '.join(launch_args)}"

print("Starting application...")
print("This may take a few moments. Please wait for the interface to load.")
print("If the interface doesn't appear, check the output below for any error messages.")

# Start the application
!{launch_command}

In [ ]:
#@title ## **Model Management**
#@markdown ### Download or manage voice models

#@markdown Action to perform:
action = "List available models" #@param ["List available models", "Download model from URL", "Upload custom model", "Delete model"]

#@markdown Model URL (for download action):
model_url = "" #@param {type:"string"}

#@markdown Model name (for delete action):
model_name = "" #@param {type:"string"}

#@markdown ---
#@markdown <br> *Run this cell to manage voice models.*

import os
import subprocess
import json
from IPython.display import clear_output, HTML, display
from pathlib import Path

# Check if installation was completed
if not Path("config.json").exists():
    print("❌ Configuration file not found. Please run the installation cell first.")
    sys.exit(1)

# Load configuration
with open("config.json", "r") as f:
    config = json.load(f)

debug_mode = config.get("debug_mode", False)

# Execute the selected action
if action == "List available models":
    result = subprocess.run("python programs/applio_code/rvc/lib/tools/list_models.py", 
                           shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    if result.returncode == 0:
        print("\nAvailable models:")
        for line in result.stdout.strip().split('\n'):
            if line:
                print(f"- {line}")
    else:
        print("Failed to list models.")

elif action == "Download model from URL":
    if not model_url:
        print("❌ Please provide a model URL.")
    else:
        print("Downloading model...")
        result = subprocess.run(f"python programs/applio_code/rvc/lib/tools/download_model.py --url {model_url}", 
                              shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
        if result.returncode == 0:
            print("✅ Model downloaded successfully!")
        else:
            print("❌ Failed to download model. Please check the URL and try again.")
            if debug_mode:
                print(result.stderr)

elif action == "Upload custom model":
    print("Please use the file upload feature in the main application to upload custom models.")
    print("You can access the file upload feature after starting the application in the previous cell.")

elif action == "Delete model":
    if not model_name:
        print("❌ Please provide a model name.")
    else:
        result = subprocess.run(f"python programs/applio_code/rvc/lib/tools/delete_model.py --name {model_name}", 
                              shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
        if result.returncode == 0:
            print(f"✅ Model '{model_name}' deleted successfully!")
        else:
            print(f"❌ Failed to delete model '{model_name}'. Please check the name and try again.")
            if debug_mode:
                print(result.stderr)

In [ ]:
#@title ## **Reset / Cleanup**
#@markdown ### Reset the environment or clean up files

#@markdown Action to perform:
reset_action = "Clear application cache" #@param ["Clear application cache", "Reset configuration", "Full reset (reinstall everything)"]

#@markdown ---
#@markdown <br> *Run this cell to perform the selected action.*

import os
import shutil
from IPython.display import clear_output
from pathlib import Path

if reset_action == "Clear application cache":
    cache_dirs = ["cache", "temp", "__pycache__"]
    for cache_dir in cache_dirs:
        if Path(cache_dir).exists():
            shutil.rmtree(cache_dir)
            print(f"✅ Cleared {cache_dir} directory")
    print("Cache cleared successfully!")

elif reset_action == "Reset configuration":
    if Path("config.json").exists():
        os.remove("config.json")
        print("✅ Configuration reset")
    else:
        print("No configuration file found")
    print("Please run the installation cell again to create a new configuration.")

elif reset_action == "Full reset (reinstall everything)":
    confirm = input("⚠️ This will delete all application files and models. Are you sure? (yes/no): ")
    if confirm.lower() == "yes":
        os.chdir("..")
        if Path("main_program").exists():
            shutil.rmtree("main_program")
            print("✅ Removed application directory")
        print("Please run the installation cell again to reinstall the application.")
    else:
        print("Reset cancelled.")